In [2]:
import numpy as np
import os
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.regularizers import l1
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

import matplotlib.pyplot as plt
import pandas as pd
from keras.utils.vis_utils import plot_model

from PIL import Image, ImageOps

import torch

from keras import models

import scipy.io

# from dfply import *

torch.manual_seed(4460)
np.random.seed(4460)

Using TensorFlow backend.


In [3]:
xtrain = scipy.io.loadmat('./data/xtrain.mat')
ytrain = scipy.io.loadmat('./data/ytrain.mat')

In [4]:
assert len(xtrain['TMRF_T2']) == len(ytrain['GS_T2'])
print('The sample size is', len(xtrain['TMRF_T2']))

The sample size is 20


In [5]:
def unet(pretrained_weights=None, input_size=(224, 224, 1)):
    inputs = Input(input_size)
    # conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    # conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    # pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    # delete the pool4 layer; instead, use a upsampling to get a up5 layer, to turn the unet
    up5 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(drop4))
    
    # the old merge 6
    merge5 = concatenate([drop3, up5], axis=3)
    # the old conv6
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    
    # to concat with conv2
    up6 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv5))
    merge6 = concatenate([conv2, up6])
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    

    conv10 = Conv2D(1, 1)(conv6)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(lr=1e-4), loss='mse',  metrics=['mae', 'mse'])
    print(model.summary())

    if (pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [6]:
random_index = np.random.choice(len(xtrain['TMRF_T2']), 10, replace=False)

xtest = xtrain['TMRF_T2'][random_index]
ytest = ytrain['GS_T2'][random_index]

xtrain = np.delete(xtrain['TMRF_T2'], random_index, axis=0)
ytrain = np.delete(ytrain['GS_T2'], random_index, axis=0)

xtrain = np.reshape(xtrain, [10, 225, 225, 1])
ytrain = np.reshape(ytrain, [10, 225, 225, 1])
xtest = np.reshape(xtest, [10, 225, 225, 1])
ytest = np.reshape(ytest, [10, 225, 225, 1])

xtrain = xtrain[:, 0:224, 0:224, :]
ytrain = ytrain[:, 0:224, 0:224, :]
xtest = xtest[:, 0:224, 0:224, :]
ytest = ytest[:, 0:224, 0:224, :]

print(len(xtrain))
print(len(xtest))

10
10


In [7]:
from keras.callbacks import EarlyStopping

# Define paths
checkpoint_path = "unet_weights_{epoch:02d}-{val_loss:.2f}.hdf5"

# Define model hyperparameters
EPOCHS = 20
SPEREPOCH = 20


# Setup definitions for display
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['loss'],
             label='Train Error')
    plt.plot(hist['epoch'], hist['mae'],
             label='Val Error')
    plt.ylim([0, 500])
    plt.legend()
    plt.show()


model = unet()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 128 1280        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 128 147584      conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 128 0           conv2d_2[0][0]                   
____________________________________________________________________________________________

/home/wq2151/.conda/envs/BMEN4460/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [ ]:
# plot_model(model, to_file='model_plot.png')

# model_checkpoint = ModelCheckpoint('unet_pepa.hdf5', monitor='loss',verbose=1, save_best_only=True)
# cp_callback =  ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1,save_best_only=True)
# cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

# cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=False,
#                               save_weights_only=True, mode='auto', period=1)

cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=False,
                              save_weights_only=True, mode='auto', period=10)

es = EarlyStopping(monitor='val_loss', mode='min', baseline=0.4)

callbacks_list = [cp_callback, es]
history = model.fit(
    xtrain, ytrain, steps_per_epoch=SPEREPOCH,
    epochs=EPOCHS, callbacks=callbacks_list, verbose=2)
print('Training done')

Epoch 1/20
